## Import libraries

In [1]:
import src.config as config

2025-06-23 11:51:22.395 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Repositories\ds-lstm-ibov


## ETL — Extract, Transform, Load of Raw Dataset

This section is responsible for the Extraction phase of the ETL process, pulling historical financial data from multiple sources as defined in the configuration file [dataset.yaml](../configs/dataset.yaml).

- Data Storage: Each dataset is stored locally in the data/raw/ directory, organized and saved individually as .csv files for traceability and versioning.
- Data Cleaning:
  - Missing values are treated using a standard cleaning strategy.
  - Features with low variance (threshold < 0.01) are removed to reduce noise and improve modeling efficiency. In this execution, no low-variance columns were found.

In [2]:
from src import dataset

dataset.main(
    # ---- REPLACE DEFAULT AS APPROPRIATE ----
    asset = '^BVSP',
    asset_focus = 'Close',
    years = 7
    # -----------------------------------------
    )

2025-06-23 11:51:25.393 | INFO     | src.dataset:main:32 - Starting raw data loading...
2025-06-23 11:51:25.393 | INFO     | src.dataset:main:38 - Requesting information between 2018-06-25 and 2025-06-23
2025-06-23 11:51:25.393 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BOVESPA (^BVSP) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-23 11:51:29.022 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading S&P500 (^GSPC) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-23 11:51:31.633 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BITCOIN (BTC-USD) from yfinance...


...

2025-06-23 11:51:39.625 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading ACUCAR (SB=F) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-23 11:51:42.292 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC (11) from the Central Bank of Brazil API...
2025-06-23 11:51:56.817 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading CDI (12) from the Central Bank of Brazil API...
2025-06-23 11:52:10.632 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Anual (1178) from the Central Bank of Brazil API...
2025-06-23 11:52:23.245 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Meta_Anual (432) from the Central Bank of Brazil API...
2025-06-23 11:52:47.093 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading IPCA_Mensal (433) from the Central Bank of Brazil API...
2025-06-23 11:52:49.678 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading IGP_M_Mensal (189) from the Central Bank of Brazil API...
2025-06-23 11:52:52.144 | INFO     | src.utils.dataset.

INFO:src.utils.dataset.clean_strategy:Executing CleanMissingValues...
INFO:src.utils.dataset.clean_strategy:Executing CleanLowVariance with threshold=0.01...
INFO:src.utils.dataset.clean_strategy:Columns removed due to low variance: Index([], dtype='object')
c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:src.utils.dataset.clean_strategy:Columns removed by f_classif: ["('Volume', 'BTC-USD')"]
c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
INFO:src.utils.dataset.clean_strategy:Columns removed by f_regression: ["('Volume', '^GSPC')"]


2025-06-23 11:53:20.633 | SUCCESS  | src.dataset:main:64 - Raw data successfully loaded...
            ('Close', '^BVSP')  ('High', '^BVSP')  ('Low', '^BVSP')  \
2025-06-17            138840.0           139497.0          138293.0   
2025-06-18            138717.0           139161.0          138443.0   
2025-06-19            138717.0           139161.0          138443.0   
2025-06-20            137116.0           138719.0          136815.0   
2025-06-21            137116.0           138719.0          136815.0   
2025-06-22            137116.0           138719.0          136815.0   
2025-06-23            137116.0           138719.0          136815.0   

            ('Open', '^BVSP')  ('Volume', '^BVSP')  ('Close', '^GSPC')  \
2025-06-17           139256.0            8377000.0         5982.720215   
2025-06-18           138844.0            8323400.0         5980.870117   
2025-06-19           138844.0            8323400.0         5980.870117   
2025-06-20           138715.0           1128

## Feature Engineering — Time Series Preparation

This part of the pipeline is responsible for transforming the cleaned dataset into structured features suitable for training time series models. It includes the following key steps:
- Feature Generation: Constructs relevant features based on historical market data.
- Dataset Splitting: The dataset is split into training and testing sets using a consistent strategy to preserve temporal structure.
- Time Series Windowing: Converts the sequential data into overlapping windows, enabling the model to learn temporal dependencies.
- Saving Artifacts: Both training and testing sets are stored for reproducibility, along with the transformation pipelines applied during preprocessing.

In [3]:
from src import config
from src import features

features.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    train_dir = config.PROCESSED_DATA_DIR,
    test_dir = config.PROCESSED_DATA_DIR,
    sequence_length = 9
    # -----------------------------------------
)

2025-06-23 11:53:24.907 | INFO     | src.features:main:32 - Generating features from dataset...
2025-06-23 11:53:24.954 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-23 11:53:24.969 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...
2025-06-23 11:53:25.014 | SUCCESS  | src.features:main:47 - Saving train features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-23 11:53:25.030 | SUCCESS  | src.features:main:51 - Saving test features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-23 11:53:25.030 | SUCCESS  | src.features:main:55 - Features generation complete.
2025-06-23 11:53:25.030 | INFO     | src.features:main:57 - Saving transformers...
2025-06-23 11:53:25.030 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-23 11:53:25.030 | INFO     | src.features:main:73 - Total t

## Modeling

The modeling process begins with loading the training dataset. Next, the base model is constructed, and both the compilation and training strategies are defined. With the pipeline structure in place, the model is trained over 100 epochs using an iterative approach to adjust the weights.

During training, key metrics are monitored, including accuracy, loss (error), validation accuracy and loss (on unseen data), as well as the learning rate.

In [4]:
from src import config
from src.modeling import train

train.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    X_path = config.PROCESSED_DATA_DIR / "X_train.npy",
    y_path = config.PROCESSED_DATA_DIR / "y_train.npy",
    batch_size = 20,
    validation_split = 0.01,
    # -----------------------------------------
)

2025-06-23 11:53:27.186 | INFO     | src.modeling.train:main:32 - Loading training dataset...
2025-06-23 11:53:27.203 | INFO     | src.modeling.train:main:42 - Building model...
2025-06-23 11:53:27.203 | INFO     | src.modeling.train:main:48 - Selecting compile strategy...
2025-06-23 11:53:27.203 | INFO     | src.modeling.train:main:51 - Selecting training strategy...
2025-06-23 11:53:27.203 | INFO     | src.modeling.train:main:54 - Building model training pipeline template...
2025-06-23 11:53:27.203 | INFO     | src.modeling.train:main:61 - Training model...
Epoch 1/1000
127/127 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.3536 - mae: 0.7046 - mse: 0.7919 - r2_score: -15.9025 - rmse: 0.7706 - smape: 134.6230 - val_loss: 0.2145 - val_mae: 0.6337 - val_mse: 0.4272 - val_r2_score: -0.7220 - val_rmse: 0.6430 - val_smape: 83.2156 - learning_rate: 0.0010
Epoch 2/1000
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.1847 - mae: 0.4842 - mse: 0.3847 - r2_score: -7.1777 - rmse: 0.5629 - smape

2025/06/23 11:59:28 INFO mlflow.tracking.fluent: Experiment with name 'lstm-ibov-experiment' does not exist. Creating a new experiment.
2025/06/23 11:59:28 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/06/23 11:59:44 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\SAMSUNG\AppData\Local\Temp\tmpuqhepxf8\model, flavor: keras). Fall back to return ['keras==3.9.2']. Set logging level to DEBUG to see the full traceback. 
2025/06/23 11:59:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'RegressionSimpleModelBuilder'.
Created version '1' of model 'RegressionSimpleModelBuilder'.


2025-06-23 11:59:57.822 | SUCCESS  | src.modeling.train:main:88 - Modeling training complete.
2025-06-23 11:59:57.823 | INFO     | src.modeling.train:main:91 - Elapsed time: 390.64 seconds


## Predict

In the prediction stage, the trained model is loaded along with the input data. The model then performs inference, generating predictions based on the provided data.


In [5]:
from src import config
from src.modeling import predict

predict.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    preprocessor_path = config.PROCESSED_DATA_DIR / "preprocessor.pkl",
    model_path = config.MODELS_DIR / "Sequential_epoch261_loss0.0273.keras",  # Select the best model  
    postprocessor_path = config.PROCESSED_DATA_DIR / "postprocessor.pkl",
    output_path = config.PROCESSED_DATA_DIR / "dataset_report.csv",
    length = 9
    # -----------------------------------------
)

2025-06-23 13:47:11.405 | INFO     | src.modeling.predict:main:29 - Performing inference for model...
2025-06-23 13:47:11.475 | INFO     | src.modeling.predict:main:33 - Input data shape: (10, 49)
2025-06-23 13:47:11.496 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...


c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 831ms/step
            ('Close', '^BVSP')  ('High', '^BVSP')  ('Low', '^BVSP')  \
2025-06-22         137116.0000           138719.0     136815.000000   
2025-06-23         137116.0000           138719.0     136815.000000   
2025-06-24         136877.9375           137529.0     136192.984375   

            ('Open', '^BVSP')  ('Volume', '^BVSP')       type  
2025-06-22      138715.000000           11289300.0       True  
2025-06-23      138715.000000           11289300.0       True  
2025-06-24      136725.890625            8417869.0  Predicted  
2025-06-23 13:47:12.821 | SUCCESS  | src.modeling.predict:main:76 - Inference complete.


## Plot

Open the [Power BI Report]((../reports/pbi/amp-fynance.pbip)) and refresh data.